####  Холодные звонки

Эффективность замены доменов на кластеры

In [1]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors

sc.stop()
conf = SparkConf().set("spark.executor.instances", 64).set("spark.driver.maxResultSize", "32g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)



In [2]:
read_query = '''
select 
    ymd, 
    ccall_approve_label as label, 
    score_ccall_total, 
    score_ccall_approve_total, 
    score_ccall_not_approve_total, 
    score_ccall_approve_ccall_not_approve, 
    score_ccall_approve_total_weekly
from user_kposminin.ccalls_scores_2
'''

read_ccalls_visits_clusters_query = '''
select 
    ymd, 
    full_app as label, 
    features
from user_kposminin.ccalls_visits_clusters m
left join user_kposminin.cold_calls_havent_started neg on neg.phone_num = m.phone_num
WHERE neg.phone_num is Null
'''

read_ccalls_visits_superclusters_query = '''
select   
  phone_num,
  -- max(label) as label,
  max(full_app) as label,
  collect_set(supercluster) as features,
  ymd
from 
  (select
    v.ymd,
    v.phone_num,
    cast(v.label as tinyint) as label,
    cast(v.full_app as tinyint) as full_app,   
  --  cast(v.visit_lag as tinyint) as visit_lag,
    cast(c.supercluster as int) as supercluster
  from user_kposminin.ccalls_visits_1 v
  inner join user_kposminin.domain_clusters c on c.domain = split(v.urlfr,'#')[0]
  left semi join user_kposminin.cold_calls_matched_5 m on m.phone_num = v.phone_num and m.ymd = v.ymd and m.havent_started = 0
  ) a
group by phone_num, ymd
'''

read_ccalls_visits_superclusters_with_count_query = '''
select 
  ymd,
  phone_num,
  max(label) as label,
  collect_list(concat(supercluster,":",cnt)) as features
from
  (select   
    phone_num,
    -- max(label) as label,
    max(full_app) as label,
    supercluster,
    stddev(visit_lag) as cnt,
    ymd
  from 
    (select
      v.ymd,
      v.phone_num,
      cast(v.label as tinyint) as label,
      cast(v.full_app as tinyint) as full_app,  
      cast(c.supercluster as int) as supercluster,
      1 as cnt,
      v.visit_lag
    from user_kposminin.ccalls_visits_1 v
    inner join user_kposminin.domain_clusters c on concat(c.domain,'#') = v.urlfr
    left semi join user_kposminin.cold_calls_matched_5 m on m.phone_num = v.phone_num and m.ymd = v.ymd and m.havent_started = 0
    ) a
  group by phone_num, ymd, supercluster
--  order by ymd,phone_num, supercluster
  ) b
group by ymd, phone_num
'''

many_features_query = '''
select 
  ymd,
  phone_num,
  max(label) as label,
  sum(cnt_uniq) as cnt_uniq,
  sum(cnt_w_repeat) as cnt_w_repeat,
  count(supercluster) as cnt_sclust,
  SUBSTR(phone_num,2,3) as phone_3digits,
  max(
       named_struct(
        'work_hours_cnt_w_repeats', work_hours_cnt_w_repeats,
        'supercluster', supercluster
       )
    ).supercluster as work_hours_sclust,
  max(
       named_struct(
        'night_hours_cnt_w_repeats', night_hours_cnt_w_repeats,
        'supercluster', supercluster
       )
    ).supercluster as night_hours_sclust,
  collect_list(
    concat(
      supercluster,","
     ,std1,","
     ,std2,","
     ,cnt_w_repeat,","
     ,cnt_uniq,","
     ,duration,","
     ,avg_hour,","
     ,work_hours_cnt_w_repeats
    )
  ) as sclust_features
from
  (select   
    phone_num,
    -- max(label) as label,
    max(full_app) as label,
    supercluster,
    stddev(visit_lag) as std1,
    stddev(-24 * visit_lag + avg_hour) as std2,
    sum(cnt) as cnt_w_repeat,
    sum(if(avg_hour >  9 and avg_hour <  20,cnt,0)) as  work_hours_cnt_w_repeats,
    sum(if(avg_hour <= 9 or  avg_hour >= 20,cnt,0)) as night_hours_cnt_w_repeats,
    sum(1) as cnt_uniq,
    sum(duration) as duration,
    cast(percentile(avg_hour,0.5) as int) as avg_hour,
    ymd
  from 
    (select
      v.ymd,
      v.phone_num,
      cast(v.label as tinyint) as label,
      cast(v.full_app as tinyint) as full_app,  
      cast(c.supercluster as int) as supercluster,
      v.cnt,
      v.visit_lag,
      v.duration,
      v.avg_hour
    from user_kposminin.ccalls_visits_1 v
    inner join user_kposminin.domain_clusters c on concat(c.domain,'#') = v.urlfr
    left semi join user_kposminin.cold_calls_matched_5 m on m.phone_num = v.phone_num and m.ymd = v.ymd and m.havent_started = 0
    ) a
  group by phone_num, ymd, supercluster
--  order by ymd,phone_num, supercluster
  ) b
group by ymd, phone_num

'''
#from scipy.sparse import coo_matrix, vstack,hstack

#tf = HashingTF(numFeatures = 10 ** 5)

all_data = hc.sql(many_features_query).rdd
train_data = all_data \
    .filter(lambda row: row['ymd'] < '2016-05-16') \
    .collect()
test_data  = all_data \
    .filter(lambda row: row['ymd'] >= '2016-05-16') \
    .collect()
    


In [37]:
#Transform into sparse matrix
from scipy.sparse import csr_matrix, vstack,hstack
phone_codes = dict((e.split('\t')[0],int(e.split('\t')[1])) for e in open('phone_codes.txt','r').read().split('\n'))

def get_sparse_matrix(data):
    labels, features = [0], csr_matrix((1,7006))
    i = 0
    batch_size = 3000
    for i in range(len(data)/batch_size + 1):
        batch = csr_matrix((1,7006))
        for r in data[batch_size*i:batch_size*(i+1)]:
            row = []
            sp3s = 1e-10
            sp4s = 1e-10
            sp5s = 1e-10
            sp7s = 1e-10
            for s in r[9]:
                sp = s.split(',')
                row.append([
                        int(sp[0]),
                        0 if sp[1] == 'NaN' else float(sp[1]),
                        0 if sp[2] == 'NaN' else float(sp[2]),
                        float(sp[3]),
                        float(sp[4]),
                        float(sp[5]),
                        int(sp[6]),
                        float(sp[7]),
                 ])
                sp3s += row[-1][3]
                sp4s += row[-1][4]
                sp5s += row[-1][5]
                sp7s += row[-1][7]
            row = [[e[0],e[1],e[2],e[3]/sp3s,e[4]/sp4s,e[5]/sp5s,e[6],e[7]/sp7s] for e in row]
            vals = [int(e) for e in r[3:9]]
            cidx = range(6)
            ridx = [0] * 6
            for s in row:
                sh = len(s) - 1
                vals += s[1:]
                cidx += [sh * s[0] + j + 6 for j in range(sh)]
                ridx += [0] * (len(s)-1)

        #idx  = [int(e.split(':')[0]) for e in r[1] if 0 <= int(e.split(':')[0]) < 1000]
        #vals = [int(e.split(':')[1]) for e in r[1] if 0 <= int(e.split(':')[0]) < 1000]
            labels.append(r.label)
            new_row = csr_matrix((vals, (ridx,cidx)), shape=(1,7006), dtype = type(1.))
            batch = vstack([batch,new_row])
        features = vstack([features, batch[1:]])
        print(str(i)+'/'+str(len(data)/batch_size + 1))
    
    return labels[1:],features[1:]



def get_sparse_matrix_2(data):
    labels, features = [0], csr_matrix((1,10100))
    i = 0
    batch_size = 3000
    for i in range(len(data)/batch_size + 1):
        k, vals, cidx, ridx = 0, [], [], []
        for r in data[batch_size*i:batch_size*(i+1)]:
            row = []
            sp3s = 0
            sp4s = 0
            sp5s = 0
            sp7s = 0
            for s in r[9]:
                sp = s.split(',')
                row.append([
                        int(sp[0]),
                        0 if sp[1] == 'NaN' else float(sp[1]),
                        0 if sp[2] == 'NaN' else float(sp[2]),
                        float(sp[3]),
                        float(sp[4]),
                        float(sp[5]),
                        int(sp[6]),
                        float(sp[7]),
                 ])
                sp3s += row[-1][3]
                sp4s += row[-1][4]
                sp5s += row[-1][5]
                sp7s += row[-1][7]
            row = [[e[0],1,e[1],e[2],e[3]/sp3s if sp3s!=0 else 0,e[4]/sp4s if sp4s!=0 else 0,e[5]/sp5s if sp5s!=0 else 0,
                    e[6],e[7]/sp7s if sp7s!=0 else 0] for e in row]
            vals += [int(e) for e in r[3:6]]
            cidx += range(3)
            ridx += [k] * 3
            for s in row:
                sh = len(s) - 1
                vals += s[1:]
                cidx += [sh * s[0] + j + 4 for j in range(sh)]
                ridx += [k] * sh
            vals += [1, 1, 1]
            cidx += [8004 + r[7], 9004 + r[8], 10004 + phone_codes[r[6]]]
            ridx += [k, k, k]
            labels.append(r.label)
            k += 1
        batch = csr_matrix((vals, (ridx,cidx)), shape=(k,10100), dtype = type(1.))
        features = vstack([features, batch])
        print(str(i)+'/'+str(len(data)/batch_size + 1))
    
    return labels[1:],features[1:]

In [82]:
train_labels, train_features = get_sparse_matrix_2(train_data)

0/152
1/152
2/152
3/152
4/152
5/152
6/152
7/152
8/152
9/152
10/152
11/152
12/152
13/152
14/152
15/152
16/152
17/152
18/152
19/152
20/152
21/152
22/152
23/152
24/152
25/152
26/152
27/152
28/152
29/152
30/152
31/152
32/152
33/152
34/152
35/152
36/152
37/152
38/152
39/152
40/152
41/152
42/152
43/152
44/152
45/152
46/152
47/152
48/152
49/152
50/152
51/152
52/152
53/152
54/152
55/152
56/152
57/152
58/152
59/152
60/152
61/152
62/152
63/152
64/152
65/152
66/152
67/152
68/152
69/152
70/152
71/152
72/152
73/152
74/152
75/152
76/152
77/152
78/152
79/152
80/152
81/152
82/152
83/152
84/152
85/152
86/152
87/152
88/152
89/152
90/152
91/152
92/152
93/152
94/152
95/152
96/152
97/152
98/152
99/152
100/152
101/152
102/152
103/152
104/152
105/152
106/152
107/152
108/152
109/152
110/152
111/152
112/152
113/152
114/152
115/152
116/152
117/152
118/152
119/152
120/152
121/152
122/152
123/152
124/152
125/152
126/152
127/152
128/152
129/152
130/152
131/152
132/152
133/152
134/152
135/152
136/152
137/152
138/15

In [83]:
test_labels, test_features = get_sparse_matrix_2(test_data)


0/34
1/34
2/34
3/34
4/34
5/34
6/34
7/34
8/34
9/34
10/34
11/34
12/34
13/34
14/34
15/34
16/34
17/34
18/34
19/34
20/34
21/34
22/34
23/34
24/34
25/34
26/34
27/34
28/34
29/34
30/34
31/34
32/34
33/34


In [88]:
import pickle
pickle.dump([train_features,train_labels,test_features,test_labels],open('train_test_data_superclust_w_all_feat.pck','w'))
#train_features,train_labels,test_features,test_labels = pickle.load(open('train_test_data_superclust_w_all_feat.pck','r'))

In [86]:
import sklearn.linear_model
mLR = sklearn.linear_model.LogisticRegression(penalty = 'l1',C = 0.01)
mLR.fit(X = train_features,y = train_labels)

print('{0} AUCROC: {1}'.format('LogRegr',sklearn.metrics.roc_auc_score(
                y_true = test_labels , 
                y_score = [e[1] for e in mLR.predict_proba(test_features)]
    )))

LogRegr AUCROC: 0.597811773318


In [46]:
import sklearn.linear_model
mLR = sklearn.linear_model.LogisticRegression(penalty = 'l1',C = 1,class_weight = 'auto')
mLR.fit(X = train_features,y = train_labels)

print('{0} AUCROC: {1}'.format('LogRegr',sklearn.metrics.roc_auc_score(
                y_true = test_labels , 
                y_score = [e[1] for e in mLR.predict_proba(test_features)]
    )))

LogRegr AUCROC: 0.595666620052


/opt/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


In [145]:
import itertools

params = [
    # C
    [0.5,1,2],
    # penalty
    ['l1','l2'],    
]

#n_iter_search = 10
aucroc = []
for C,penalty in itertools.product(*params):
    clf = sklearn.linear_model.LogisticRegression(penalty = penalty,C = C,class_weight = 'auto')
    clf.fit(X = train_sampled_features,y = train_sampled_labels)
    aucroc.append([(C,penalty),sklearn.metrics.roc_auc_score(
                y_true = test_labels , 
                y_score = [e[1] for e in clf.predict_proba(test_features)]
    )])

print(aucroc)


[[(0.5, 'l1'), 0.59098475338660805], [(0.5, 'l2'), 0.5917140064965497], [(1, 'l1'), 0.59257132119265288], [(1, 'l2'), 0.5923741801180924], [(2, 'l1'), 0.59118285580880081], [(2, 'l2'), 0.59219354829882509]]


In [47]:
sample_index = [i for i in range(len(train_labels)) if train_labels[i] == 1 or np.random.randint(0,4) == 0]

In [48]:
train_sampled_labels = [train_labels[e] for e in sample_index]
train_sampled_features = train_features[sample_index]

In [146]:
import sklearn.linear_model
mLR = sklearn.linear_model.LogisticRegression(penalty = 'l1',C = 1, class_weight = 'auto')
mLR.fit(X = train_sampled_features,y = train_sampled_labels)

print('{0} AUCROC: {1}'.format('LogRegr',sklearn.metrics.roc_auc_score(
                y_true = test_labels , 
                y_score = [e[1] for e in mLR.predict_proba(test_features)]
    )))

LogRegr AUCROC: 0.592571391043


In [106]:
from sklearn import linear_model
lin_models = {
    'Ridge': linear_model.Ridge (alpha = .5),
    'Lasso': linear_model.Lasso(),
    'ElasticNet': linear_model.ElasticNet(),
    'Lars': linear_model.Lars(),
    'LinRegr': linear_model.LinearRegression(),
    'BayesianRidge':  linear_model.BayesianRidge()
}
for m in lin_models:
    lin_models[m].fit(X = train_features.toarray(),y = train_labels)
    print('{0} AUCROC:    {1}'.format(m,sklearn.metrics.roc_auc_score(
                y_true =  test_labels , 
                y_score = lin_models[m].predict(test_features.toarray()))))

Lars AUCROC:    0.593023390056
LinRegr AUCROC:    0.590445436645
Ridge AUCROC:    0.591382633208
ElasticNet AUCROC:    0.5
BayesianRidge AUCROC:    0.593445548406
Lasso AUCROC:    0.5


In [87]:
import sklearn.ensemble
#from sklearn import svm
clf = {
   # 'LinSVC': sklearn.svm.LinearSVC(penalty='l1', loss='squared_hinge',C=1.0, class_weight='auto', max_iter=1000),
   #'SVC': sklearn.svm.SVC(probability = True,max_iter = 100),
   # 'LogRegr': sklearn.linear_model.LogisticRegression(),
    'RandomForest': sklearn.ensemble.RandomForestClassifier(max_depth = 5,n_estimators = 50),
   # 'GBM': sklearn.ensemble.GradientBoostingClassifier(n_estimators = 400)
    
}
for m in clf:
    clf[m].fit(X = train_sampled_features,y = train_sampled_labels)
    print('{0} AUCROC: {1}'.format(m,sklearn.metrics.roc_auc_score(
                y_true = test_labels, 
                y_score = [e[1] for e in clf[m].predict_proba(test_features)]
    )))

RandomForest AUCROC: 0.593976064676


In [76]:
[(v-4) % 8 for v in (sorted([e[1] for e in sorted(zip(clf['RandomForest'].feature_importances_,range(len(clf['RandomForest'].feature_importances_))),reverse = True)[:31]]))]



[7,
 3,
 4,
 5,
 7,
 4,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 6,
 7,
 2,
 4,
 7,
 0,
 2,
 3,
 7,
 0,
 4,
 5,
 7,
 6,
 7,
 0,
 1,
 3]

In [61]:
clfXGB.

<bound method XGBClassifier.get_xgb_params of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)>

In [49]:
import xgboost as xgb
clfXGB = xgb.XGBClassifier()
clfXGB.fit(X = train_features,y = train_labels)
print('{0} AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = test_labels, 
                y_score = [e[1] for e in clfXGB.predict_proba(test_features)]
    )))

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [50]:
print('{0} AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = test_labels, 
                y_score = [e[1] for e in clfXGB.predict_proba(test_features)]
    )))

XGBoost AUCROC: 0.606434786497


In [59]:
#sorted(zip(clfXGB.feature_importances_,range(len(clfXGB.feature_importances_))),inverse = True)[:30]


<bound method XGBClassifier.score of XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)>

In [34]:
from sklearn.ensemble import AdaBoostClassifier

clfAB = AdaBoostClassifier()
clfAB.fit(X = train_features,y = train_labels)

print('{0} AUCROC: {1}'.format('AdaBoost',sklearn.metrics.roc_auc_score(
                y_true = test_labels, 
                y_score = [e[1] for e in clfAB.predict_proba(test_features)]
    )))

AdaBoost AUCROC: 0.602660247649


In [54]:
len(clfAB.feature_importances_)

NameError: name 'clfAB' is not defined

In [26]:
import sys
import math
 
import numpy as np
from sklearn.grid_search import GridSearchCV
 
sys.path.append('xgboost/wrapper/')
import xgboost as xgb
 
 
class XGBoostClassifier():
    def __init__(self, num_boost_round=10, **params):
        self.clf = None
        self.num_boost_round = num_boost_round
        self.params = params
        self.params.update({'objective': 'multi:softprob'})
 
    def fit(self, X, y, num_boost_round=None):
        num_boost_round = num_boost_round or self.num_boost_round
        self.label2num = dict((label, i) for i, label in enumerate(sorted(set(y))))
        dtrain = xgb.DMatrix(X, label=[self.label2num[label] for label in y])
        self.clf = xgb.train(params=self.params, dtrain=dtrain, num_boost_round=num_boost_round)
 
    def predict(self, X):
        num2label = dict((i, label)for label, i in self.label2num.items())
        Y = self.predict_proba(X)
        y = np.argmax(Y, axis=1)
        return np.array([num2label[i] for i in y])
 
    def predict_proba(self, X):
        dtest = xgb.DMatrix(X)
        return self.clf.predict(dtest)
 
    def score(self, X, y):
        Y = self.predict_proba(X)
        return 1 / logloss(y, Y)
 
    def get_params(self, deep=True):
        return self.params
 
    def set_params(self, **params):
        if 'num_boost_round' in params:
            self.num_boost_round = params.pop('num_boost_round')
        if 'objective' in params:
            del params['objective']
        self.params.update(params)
        return self
    
    
def logloss(y_true, Y_pred):
    label2num = dict((name, i) for i, name in enumerate(sorted(set(y_true))))
    return -1 * sum(math.log(y[label2num[label]]) if y[label2num[label]] > 0 else -np.inf for y, label in zip(Y_pred, y_true)) / len(Y_pred)


def main():
    clf = XGBoostClassifier(
        eval_metric = 'auc',
        num_class = 2,
        nthread = 4,
        eta = 0.1,
        num_boost_round = 80,
        max_depth = 12,
        subsample = 0.5,
        colsample_bytree = 1.0,
        silent = 1,
        )
    parameters = {
        'num_boost_round': [100, 250, 500],
        'eta': [0.05, 0.1, 0.3],
        'max_depth': [6, 9, 12],
        'subsample': [0.9, 1.0],
        'colsample_bytree': [0.9, 1.0],
    }
    clf = GridSearchCV(clf, parameters, n_jobs=1, cv=2)
    
    clf.fit([[1,2], [3,4], [2,1], [4,3]], ['a', 'b', 'a', 'b'])
    best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
    print(score)
    for param_name in sorted(best_parameters.keys()):
        print("%s: %r" % (param_name, best_parameters[param_name]))
                
    print(clf.predict([[1,2]]))

{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 0,
 'silent': True,
 'subsample': 1}

In [93]:
from sklearn.linear_model import SGDClassifier
import sklearn.neighbors
clf = {
   # 'SVC': sklearn.svm.SVC(probability = True,max_iter = 100),
   # 'LogRegr': sklearn.linear_model.LogisticRegression(),
    'SGD': sklearn.linear_model.SGDClassifier(penalty = 'elasticnet',loss = 'log',class_weight = 'auto'),
    'Nearest centriod': sklearn.neighbors.NearestCentroid(),
   # 'GBM': sklearn.ensemble.GradientBoostingClassifier(n_estimators = 400)
    
}
for m in clf:
    clf[m].fit(X = train_features,y = train_labels)
    print('{0} AUCROC: {1}'.format(m,sklearn.metrics.roc_auc_score(
                y_true = test_labels, 
                y_score = clf[m].predict(test_features)
    )))


SGD AUCROC: 0.540228960601
Nearest centriod AUCROC: 0.522471749342


In [108]:
import sklearn.naive_bayes

clf = {
   # 'SVC': sklearn.svm.SVC(probability = True,max_iter = 100),
   # 'LogRegr': sklearn.linear_model.LogisticRegression(),
    'NaiveBayes Multinomial': sklearn.naive_bayes.MultinomialNB(alpha = 0.001),
    'NaiveBayes Bernoulli': sklearn.naive_bayes.BernoulliNB(alpha = 0.01),
    'NaiveBayes Gaussian': sklearn.naive_bayes.GaussianNB(),
   # 'GBM': sklearn.ensemble.GradientBoostingClassifier(n_estimators = 400)
    
}
for m in clf:
    clf[m].fit(X = train_sampled_features.toarray(),y = train_sampled_labels)
    print('{0} AUCROC: {1}'.format(m,sklearn.metrics.roc_auc_score(
                y_true = test_labels, 
                y_score = [e[1] for e in clf[m].predict_proba(test_features.toarray())]
    )))


NaiveBayes Multinpomial AUCROC: 0.583131827909
NaiveBayes Bernoulli AUCROC: 0.572231601103
NaiveBayes Gaussian AUCROC: 0.517012349753


In [147]:
import sklearn.neural_network

clf = {
   # 'SVC': sklearn.svm.SVC(probability = True,max_iter = 100),
   # 'LogRegr': sklearn.linear_model.LogisticRegression(),
    'neural_network.BernoulliRBM': sklearn.neural_network.BernoulliRBM(),
   # 'NaiveBayes Bernoulli': sklearn.naive_bayes.BernoulliNB(alpha = 0.01),
   # 'NaiveBayes Gaussian': sklearn.naive_bayes.GaussianNB(),
   # 'GBM': sklearn.ensemble.GradientBoostingClassifier(n_estimators = 400)
    
}
for m in clf:
    clf[m].fit(X = train_sampled_features,y = train_sampled_labels)
    print('{0} AUCROC: {1}'.format(m,sklearn.metrics.roc_auc_score(
                y_true = test_labels, 
                y_score = [e[1] for e in clf[m].predict_proba(test_features.toarray())]
    )))


AttributeError: 'BernoulliRBM' object has no attribute 'predict_proba'